In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Crear la sesión de Spark
spark = SparkSession.builder.appName("data_read_write").getOrCreate()

# Crear un esquema para los datos
schema = StructType([
    StructField("latitude", DoubleType(), True),
    StructField("longitude", DoubleType(), True),
    StructField("date", StringType(), True),
    StructField("customer_id", IntegerType(), True),
    StructField("employee_id", IntegerType(), True),
    StructField("quantity_products", IntegerType(), True),
    StructField("order_id", StringType(), True),
])

def receive_data(host, port, interval):
    try:

        # Leer los datos del socket
        raw_data = spark.readStream.format("socket").option("host", host).option("port", port).load()

        # Convertir los datos a DataFrame usando el esquema
        json_df = raw_data.select(from_json(col("value").cast("string"), schema).alias("data")).select("data.*")
    
        json_df = json_df.repartition(1)

        query = json_df.writeStream \
                .format("parquet") \
                .outputMode("append") \
                .option("path", f"/datalake/raw/stagging") \
                .option("checkpointLocation", "/checkpoints") \
                .trigger(processingTime=interval) \
                .start()
        query.awaitTermination()
            
    except Exception as e:
        print(f"Error: {e}")
    except KeyboardInterrupt:
        print("Deteniendo lectura...")
        query.stop()
    finally:
        query.stop()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
#receive_data("30 minutes")
receive_data('localhost', 4224, "30 seconds")

KeyboardInterrupt: 